In [2]:
import requests
from selectolax.parser import HTMLParser

import json

In [21]:
def get_html():
    url = "https://www.wikidex.net/wiki/Lista_de_habilidades"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
    }
    
    data = requests.get(url)
    html = HTMLParser(data.text)
    return html


def extract(html, css_selector, option='text'):
    element = html.css_first(css_selector)
    
    if not element == None:
        if option == 'text':
            return element.text(strip=True)
        elif option == 'attrs':
            return element.attributes
        elif option == 'title':
            return element.attributes[option]
        else:
            return element
        
def save_json(data, filename):
    with open(filename, 'w', encoding="UTF-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [30]:
html = get_html()
rows = html.css(".tabpokemon tbody tr")[1:]

permitted_gens = ['Tercera generación', 'Cuarta generación', 'Quinta generación', 'Sexta generación']
translations = {}

for row in rows:
    columns = row.css("td")

    ability = {
        "spa": extract(columns[1], "a"),
        "eng": extract(columns[1], "i"),
        "gen": extract(columns[3], "a", 'title')
    }
    if ability['gen'] in permitted_gens:
        translations[ability['spa']] = ability['eng']
        

print(f"Done {len(translations)} translations.")
save_json(translations, "translations_abilities.json")

Done 192 translations.
